In [7]:
import pandas as pd
from constraint import Problem, AllDifferentConstraint

In [8]:
muscle_map_df = pd.read_csv('muscle_map.csv')
# I want to get all the distinct values from column Body Part
muscle_map_df['Muscle'] = muscle_map_df['Muscle'].apply(lambda x: x.lower())
# rename values with Thigs with Thighs
muscle_map_df['Body Part'] = muscle_map_df['Body Part'].apply(lambda x: x.replace('Thigs', 'Thighs'))
muscle_map_df['Body Part'] = muscle_map_df['Body Part'].apply(lambda x: x.replace('neck', 'Neck'))
muscle_map_df.head()

,Muscle,Body Part
0,hamstrings (below knees),Calves
1,"erector spinae, cervicis & capitis fibers",Back
2,triceps brachii,Upper Arms
3,gracilis,Thighs
4,"pectoralis major, sternal",Chest


In [9]:
muscle_map_df['Body Part'].unique()

array(['Calves', 'Back', 'Upper Arms', 'Thighs', 'Chest', 'Hips',
       'Forearms', 'Shoulders', 'Waist', 'Neck'], dtype=object)

In [12]:
def useless_replacement(x):
     x = x.lower()
     x = x.replace("none", "").replace("no significant stabilizers", "").replace("see comments", "").replace("see comment", "")
     x =  x.replace("(neck)","neck").replace("see notes","").replace("see note","").replace("\xa0","").replace("other","")
     return x.replace("( )","").replace("()","").replace(" ","").title()

In [13]:
exercises_df = pd.read_csv('exrx.csv')
# Transform all the target muscles into a single string in lower case. It is a string array by default
exercises_df['target_muscles'] = exercises_df['target_muscles'].apply(lambda x: x[2:-2].lower())
# # Convert all muscle names to lowercase in the mapping dictionary
muscle_map_dict = dict(zip(muscle_map_df['Muscle'], muscle_map_df['Body Part']))

# Use map and lambda to create the 'major muscle' column in 'exercises_df'
exercises_df['minor_muscle'] = exercises_df['target_muscles'].apply(lambda x: muscle_map_dict.get(x, 'Unknown'))
#remove the rows with 'Unknown' major muscle
exercises_df = exercises_df[exercises_df['minor_muscle'] != 'Unknown']
#remove the rows with 'Specialized' utility
exercises_df = exercises_df[exercises_df['utility'] != 'Specialized']
#remove the rows with Null utility
exercises_df = exercises_df[exercises_df['utility'].notnull()]
# Make a dict that has Legs , Chest, Back, Arms and Shoulders as keys and the values are the exercises that belong to that muscle group
muscle_group_dict = {'Legs':['Thighs','Hips','Calves','Waist'] , 'Chest':['Chest'], 'Back':['Back'], 'Arms':['Forearms','Upper Arms'] ,'Shoulders':['Neck','Shoulders']}
# Make a column major_muscle that is based on the value from minor_muscle column and use the dict to map it to the major muscle group
exercises_df['major_muscle'] = exercises_df['minor_muscle'].apply(lambda x: [k for k,v in muscle_group_dict.items() if x in v][0])
# Remove rows with duplicate exercise names
exercises_df = exercises_df.drop_duplicates(subset=['exercise'])
exercises_df = exercises_df.drop(columns=['force'])
# replace None string with "" in stablizer_muscles column
exercises_df['stabilizer_muscles'] = exercises_df['stabilizer_muscles'].apply(lambda x: useless_replacement(x))
# Repeat the same process as above for columns synergist_muscles,dynamic_stabilizer_muscles,antagonist_stabilizer_muscles
exercises_df['synergist_muscles'] = exercises_df['synergist_muscles'].apply(lambda x: useless_replacement(x))
exercises_df['dynamic_stabilizer_muscles'] = exercises_df['dynamic_stabilizer_muscles'].apply(lambda x: useless_replacement(x))
exercises_df['antagonist_stabilizer_muscles'] = exercises_df['antagonist_stabilizer_muscles'].apply(lambda x: useless_replacement(x))

exercises_df['utility'] = exercises_df['utility'].apply(lambda x: 'Basic or Auxiliary' if x == 'Auxiliary or Basic' else x)
exercises_df.head()

,exercise,utility,mechanics,target_muscles,synergist_muscles,stabilizer_muscles,dynamic_stabilizer_muscles,antagonist_stabilizer_muscles,minor_muscle,major_muscle
0,Barbell Triceps Extension,Basic or Auxiliary,Isolated,triceps brachii,[''],"['Deltoid,Anterior','Pectoralismajor,Clavicula...",[''],[''],Upper Arms,Arms
1,Cable Bar Lying Triceps Extension,Basic,Isolated,triceps brachii,[''],"['Deltoid,Anterior','Pectoralismajor,Clavicula...",[''],[''],Upper Arms,Arms
2,Smith Rear Delt Row,Basic or Auxiliary,Compound,"deltoid, posterior","['Infraspinatus','Teresminor','Deltoid,Lateral...","['Erectorspinae','Hamstrings','Gluteusmaximus'...",['Bicepsbrachii'],"['Rectusabdominis','Obliques']",Shoulders,Shoulders
3,Lever Hack Squat (plate loaded),Basic or Auxiliary,Compound,gluteus maximus,"['Quadriceps','Adductormagnus','Soleus']",[''],"['Hamstrings','Gastrocnemius']",[''],Hips,Legs
4,Sled Lying Reverse Calf Press (plate loaded),Basic or Auxiliary,Isolated,tibialis anterior,[''],[''],[''],[''],Calves,Legs


In [14]:
# find rows that have compound mechanics and auxiliary utility
exercises_df[(exercises_df['mechanics'] == 'Compound') & (exercises_df['utility'] == 'Auxiliary')]

,exercise,utility,mechanics,target_muscles,synergist_muscles,stabilizer_muscles,dynamic_stabilizer_muscles,antagonist_stabilizer_muscles,minor_muscle,major_muscle
17,Barbell Lateral Step-up,Auxiliary,Compound,quadriceps,"['Gluteusmaximus','Adductormagnus','Soleus','G...","['Erectorspinae','Gluteusmedius','Gluteusminim...","['Hamstrings','Gastrocnemius(Leadleg)']","['Rectusabdominis','Obliques']",Thighs,Legs
27,Suspended Power Pull,Auxiliary,Compound,"back, general","['Trapezius,Middle','Trapezius,Lower','Rhomboi...","['Erectorspinae','Gluteusmaximus','Hamstrings']","['Bicepsbrachii','Triceps,Longhead']",['Rectusabdominis'],Back,Back
28,Cable Split Squat (belt),Auxiliary,Compound,gluteus maximus,"['Quadriceps','Adductormagnus','Soleus']",[''],"['Hamstrings','Gastrocnemius']",[''],Hips,Legs
53,Barbell Walking Lunge,Auxiliary,Compound,gluteus maximus,"['Quadriceps','Adductormagnus','Soleus']","['Erectorspinae','Tibialisanterior','Gluteusme...","['Hamstrings','Gastrocnemius']",[''],Hips,Legs
54,Barbell Side Split Squat,Auxiliary,Compound,gluteus maximus,"['Quadriceps','Adductormagnus(Leadleg)','Adduc...","['Erectorspinae','Gluteusmedius','Gluteusminim...","['Hamstrings','Gastrocnemius']",[''],Hips,Legs
...,...,...,...,...,...,...,...,...,...,...
1279,Cable One Arm Chest Press,Auxiliary,Compound,"pectoralis major, sternal","['Pectoralismajor,Clavicular','Deltoid,Anterio...",[''],"['Bicepsbrachii,Shorthead']",[''],Chest,Chest
1320,Barbell Board Press,Auxiliary,Compound,"pectoralis major, sternal","['Pectoralismajor,Clavicular','Deltoid,Anterio...","['Erectorspinae','Gluteusmaximus']","['Bicepsbrachii,Shorthead']",[''],Chest,Chest
1376,Cable One Arm Standing Incline Chest Press,Auxiliary,Compound,"pectoralis major, clavicular","['Pectoralismajor,Sternal','Deltoid,Anterior',...",['Rectusabdominis'],"['Bicepsbrachii,Shorthead']",['Erectorspinae'],Chest,Chest
1397,Lever Single Leg Standing Leg Press (plate loa...,Auxiliary,Compound,quadriceps,"['Gluteusmaximus','Adductormagnus']","['Erectorspinae','Obliques','Gluteusmedius','G...",[''],"['Hamstrings','Rectusabdominis']",Thighs,Legs


In [15]:
# output the dataframe to a csv file
exercises_df.to_csv('exercises_cleaned.csv', index=False)